In [1]:
import pandas as pd
import numpy as np
import contractions
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
data=pd.read_csv('train.csv',encoding_errors='ignore')

In [3]:
data.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km),Density (P/Km)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [4]:
data.drop(columns=['textID','selected_text','Time of Tweet','Age of User','Country','Population -2020','Land Area (Km)','Density (P/Km)'],inplace=True)

In [5]:
data.head()

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


In [6]:
slangs="""AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My Ass| Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The Ass
PRT=Party
PRW=Parents Are Watching
QPSA?=	Que Pasa?
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My Ass Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=TaTa For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The Fuck
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait
7K=Sick Laugher
TFW = That feeling when. TFW internet slang often goes in a caption to an image.
MFW = My face when
MRW = My reaction when
IFYP = I feel your pain
LOL = Laughing out loud
TNTL = Trying not to laugh
JK = Just kidding
IDC = I don’t care
ILY = I love you
IMU = I miss you
ADIH = Another day in hell
IDC = I don’t care
ZZZ = Sleeping, bored, tired
WYWH = Wish you were here
TIME = Tears in my eyes
BAE = Before anyone else
FIMH = Forever in my heart
BSAAW = Big smile and a wink
BWL = Bursting with laughter
LMAO = Laughing my ass off
BFF= Best friends forever
CSL = Can’t stop laughing"""

In [7]:
slang_dict=dict(line.split("=", 1) for line in slangs.lower().strip().split("\n"))

In [8]:
slang_dict

{'afaik': 'as far as i know',
 'afk': 'away from keyboard',
 'asap': 'as soon as possible',
 'atk': 'at the keyboard',
 'atm': 'at the moment',
 'a3': 'anytime, anywhere, anyplace',
 'bak': 'back at keyboard',
 'bbl': 'be back later',
 'bbs': 'be back soon',
 'bfn': 'bye for now',
 'b4n': 'bye for now',
 'brb': 'be right back',
 'brt': 'be right there',
 'btw': 'by the way',
 'b4': 'before',
 'cu': 'see you',
 'cul8r': 'see you later',
 'cya': 'see you',
 'faq': 'frequently asked questions',
 'fc': 'fingers crossed',
 'fwiw': "for what it's worth",
 'fyi': 'for your information',
 'gal': 'get a life',
 'gg': 'good game',
 'gn': 'good night',
 'gmta': 'great minds think alike',
 'gr8': 'great!',
 'g9': 'genius',
 'ic': 'i see',
 'icq': 'i seek you (also a chat program)',
 'ilu': 'ilu: i love you',
 'imho': 'in my honest/humble opinion',
 'imo': 'in my opinion',
 'iow': 'in other words',
 'irl': 'in real life',
 'kiss': 'keep it simple, stupid',
 'ldr': 'long distance relationship',
 'lm

In [9]:
lemma=WordNetLemmatizer()

In [10]:
stop_words=stopwords.words('English')

In [11]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [12]:
def preprocessing(text):
    text = re.sub(r'\d+', '<NUM>', text)
    url_regex = re.compile(r"http[s]?://\S+|www\.\S+")
    text=url_regex.sub('', text)
    text=text.replace('`',"'")
    text=contractions.fix(text).lower()
    tokens=[]
    for item in nltk.word_tokenize(text):
        if item not in string.punctuation:
            tokens.append(item)
    text=" ".join(tokens)
    tokens.clear()
    for item in text.split():
        if item in slang_dict.keys():
           text=text.replace(item,slang_dict.get(item))
    for item in text.split():
        tokens.append(lemma.lemmatize(item,pos='v'))
    text=" ".join(tokens)
    tokens.clear()
    for item in nltk.word_tokenize(text):
        if item not in stop_words:
            tokens.append(item)
        text=" ".join(tokens)
    return text

In [13]:
data.isnull().sum()

text         1
sentiment    0
dtype: int64

In [14]:
data[data['text'].isnull()]

,text,sentiment
314,NaN,neutral


In [15]:
data.dropna(inplace=True)

In [16]:
data['text']=data['text'].apply(preprocessing)

In [17]:
data.sample(10)

,text,sentiment
6315,home phone die thou,negative
3044,love paramore welcome back,positive
4069,thank ballroom dance competition,positive
5030,set excite sunday laugh loud,positive
14974,hey boys want see teeth twitter donï¿½t want c...,neutral
20555,drink num num club drink past num hours,neutral
6649,rain calicut,neutral
14860,feelin depress miss soo fcking much besties .....,negative
26933,best night,positive
21405,home exactly mean ... home d-town also compute...,negative


In [18]:
data.head()

,text,sentiment
0,would respond go,neutral
1,sooo sad miss san diego,negative
2,boss bully ...,negative
3,interview leave alone,negative
4,sons could put release already buy,negative


In [19]:
data=data[data['text']!=""]

In [20]:
data.shape

(27398, 2)

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC,SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier,StackingClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder,FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [22]:
tfidf=TfidfVectorizer(max_features=18000, stop_words='english', ngram_range=(1,2), sublinear_tf=True)

In [23]:
tfidf.fit(data['text'])

TfidfVectorizer(max_features=18000, ngram_range=(1, 2), stop_words='english',
                sublinear_tf=True)

In [24]:
len(tfidf.vocabulary_)

18000

In [25]:
X=tfidf.fit_transform(data['text']).toarray()

In [26]:
encoder=LabelEncoder()
data['sentiment']=encoder.fit_transform(data['sentiment'])

In [27]:
X_train,X_test,y_train,y_test=train_test_split(X,data['sentiment'],test_size=0.2)

In [28]:
sgd_classifier=SGDClassifier(loss="log_loss", penalty="l2", alpha=1e-5, max_iter=2000, tol=1e-4, n_jobs=-1)

In [29]:
nb_classifier=MultinomialNB(alpha=0.1, fit_prior=True)

In [30]:
lr_classifier=LogisticRegression(solver="saga", penalty="l2", C=5, max_iter=20000, tol=1e-4, n_jobs=-1)

In [31]:
nb_classifier.fit(X_train,y_train)

MultinomialNB(alpha=0.1)

In [32]:
lr_classifier.fit(X_train,y_train)

LogisticRegression(C=5, max_iter=20000, n_jobs=-1, solver='saga')

In [33]:
sgd_classifier.fit(X_train,y_train)

SGDClassifier(alpha=1e-05, loss='log_loss', max_iter=2000, n_jobs=-1,
              tol=0.0001)

In [34]:
lr_prediction=lr_classifier.predict(X_test)

In [35]:
nb_prediction=nb_classifier.predict(X_test)

In [36]:
sgd_prediction=sgd_classifier.predict(X_test)

In [37]:
from sklearn.model_selection import cross_val_score

In [38]:
print(accuracy_score(lr_prediction,y_test))
print(accuracy_score(nb_prediction,y_test))
print(accuracy_score(sgd_prediction,y_test))

0.6649635036496351
0.6010948905109489
0.6781021897810219


In [39]:
def predict_new(new_text):
    processed_text=preprocessing(new_text)
    processed_text=tfidf.transform([processed_text]).toarray()
    pred=lr_classifier.predict(processed_text)
    return encoder.classes_[pred]

In [40]:
new_text="Hahaha! Alright  .."

In [41]:
predict_new(new_text)

array(['positive'], dtype=object)

In [42]:
import dill

In [43]:
import pickle

In [44]:
pickle.dump(tfidf,open('vectorizer.pkl','wb'))

In [45]:
pickle.dump(lr_classifier,open('classifier.pkl','wb'))

In [46]:
dill.dump(slang_dict,open('slangs.pkl','wb'))

In [47]:
dill.dump(preprocessing,open('preprocessing.pkl','wb'))

In [48]:
pickle.dump(encoder,open('encoder.pkl','wb'))